# 0.0 Imports

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import math
import inflection

## 0.1 Helper Functions

## 0.2 Loading data

In [2]:
#dataset que contém dados das vendas
df_sales_raw = pd.read_csv('../data/train.csv', low_memory=False)

#dataset que contém dados das lojas
df_stores_raw = pd.read_csv('../data/store.csv', low_memory=False)

#dataset final
df_raw = pd.merge(df_sales_raw, df_stores_raw, how='left', on='Store')

In [3]:
df_sales_raw.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
df_stores_raw.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
df_raw.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# 1.0 Descrição dos dados

## Qual é a motivação por trás dessa tarefa?

Descobrir o quão desafiador é o problema.

* É preciso saber a quantidade de dados:
    * eu tenho recursos corretos para trabalhar?
    * preciso de servidores? clusters? ferramentas para trabalhar com esses dados, como spark e hadoop?
    
* Tipos de variáveis:
    * quais os tipos de variáveis? % de numéricas, categóricas e temporais.
    
* Quantidade de dados faltantes:
    * qual o volume de dados NA?
    * qual a razão de faltar dados? é o sistema? input manual?
    
* Resumo geral dos dados:
    * estatística descritiva - importante para ter uma noção dos dados. dos limites máximos e mínimos..

## Data Dictionary

* Id - an Id that represents a (Store, Date) duple within the test set
* Store - a unique Id for each store
* Sales - the turnover for any given day (this is what you are predicting)
* Customers - the number of customers on a given day
* Open - an indicator for whether the store was open: 0 = closed, 1 = open
* StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* StoreType - differentiates between 4 different store models: a, b, c, d
* Assortment - describes an assortment level: a = basic, b = extra, c = extended
* CompetitionDistance - distance in meters to the nearest competitor store
* CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* Promo - indicates whether a store is running a promo on that day
* Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

## 1.1 Rename Columns

In [6]:
df1 = df_raw.copy()

É importante para criar um padrão nos nomes da coluna, além de renomear colunas que possuem nomes que não fazem muito sentido ou que são muito extensos. Isso facilita todo o desenvolvimento. Nesse ambiente do Kaggle e de datasets de estudo, as colunas já vem melhor nomeadas. Mas no contexto real nem sempre é assim. Por isso é uma boa prática realizar essa etapa.

In [7]:
#colunas antigas
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 
            'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

#função para alterar o nome das colunas para o formato snakecase
change_to_snakecase = lambda x: inflection.underscore(x)

#aplicando a função nas colunas antigas
new_cols = list(map(change_to_snakecase, cols_old))

#renomeando as colunas
df1.columns = new_cols

#verificando os novos nomes
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2 Data dimension

In [8]:
shape = df1.shape
print(f'Number of rows: {shape[0]}')
print(f'Number of columns: {shape[1]}')

Number of rows: 1017209
Number of columns: 18


## 1.3 Data types

In [9]:
#alterando o tipo da coluna date para datetime (estava como object)
df1['date'] = pd.to_datetime(df1['date'])

df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Verificação de dados faltantes

In [10]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5 Substituição dos dados faltantes

Existem basicamente três maneiras para lidar com dados faltantes:
1. Eliminar as linhas que possuem NA
    * Vantagem: rápido e fácil
    * Desvantagem: pode desperdiçar dados
    
    
2. Substituir os faltantes por métricas (média, mediana...) ou por previsão de algoritmos


3. Conhecendo do negócio e entendendo por que esses NAs estão aí


##### competition_distance

É a distância em metros até a loja concorrente mais próxima.

* Hipótese: se esse valor está nulo, é porque não há competidor próximo. O concorrente está muito longe.

  Assumindo essa hipótese, substituirei os nulos dessa coluna por um 3x maior que o maior valor da coluna. 

In [ ]:
max_value = df1['competition_distance'].max()
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: max_value*3 if math.isnan(x) else x)

#### competition_open_since_month & competition_open_since_year 

Dá aproximadamente o ano e o mês em que o competidor mais próximo foi aberto.

Pensando no negócio, essa coluna pode estar nula basicamente por duas razões:
1. Não existe um competidor próximo.
2. Existe um competidor mas alguém esqueceu de anotar, ou porque abriu muito antes ou muito depois.

Hipótese: Irei optar por manter essa coluna pois acredito que ela deve inlfuenciar nos resultados, já que o fato de uma loja possuir um competidor próximo ou não impacta diretamente nas vendas. Ainda mais nos casos em que temos um valor preenchido para 'competition_distance', não faria sentido colocar 0 para a 'competition_open_since_month' e 'competition_open_since_year', já que está sendo afirmado que existe um competidor.

Para preencher a coluna, então, pegarei o mês e o ano da linha em questão.

In [32]:
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

#### promo2_since_week & promo2_since_year

Descreve o ano e a semana do calendário quando a loja começou a participar da Promo2. A Promo2 é a continuação de uma primeira promoção (0 se não participar dessa continuação e 1 se participou).

Para a substituição dos faltantes nessa coluna, seguirei a mesma lógica das colunas 'competition_open_since_month' e 'competition_open_since_year'.

In [33]:
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

#### promo_interval              

Descreve os intervalos consecutivos que a Promo2 começou, nomeando os meses que a promoção ficou ativa. Por exemplo, "Feb, May, Aug, Nov" significa que cada round começou em Fevereiro, depois de novo em Maio, Agosto e Novembro do ano descrito para a loja específica.

* Caso o campo dessa coluna esteja nulo, será substituído por 0, indicando que não foi feita continuação da Promo2.

In [34]:
#diicionário para mapear os meses
month_map = {
    1:'Jan',
    2:'Feb',
    3:'Mar',
    4:'Apr',
    5:'May',
    6:'Jun',
    7:'Jul',
    8:'Aug',
    9:'Sep',
    10:'Oct',
    11:'Nov',
    12:'Dec'
}

#preenche os nulos com 0
df1['promo_interval'].fillna(0, inplace=True)

#cria uma coluna 'month_map' com o nome dos meses
df1['month_map'] = df1.date.dt.month.map(month_map)

#cria uma coluna 'is_promo' que indica se no mês do registro teve Promo2 ou não
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

## 1.6 Estatística Descritiva 

In [52]:
df1.head()

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,31.0,2015.0,0,Jul,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",Jul,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",Jul,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,31.0,2015.0,0,Jul,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,31.0,2015.0,0,Jul,0
